In [ ]:
import numpy as np
import pandas as pd
import os

def consecutive_bool(data, stepsize=1):
    return np.split(data, np.where(np.diff(data) != stepsize)[0]+1)

# set manual parameters
window_pts = 70
len_lim = 3
dt_loc_thresh = 550
loc_it = 4

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

Resave MINFLUX datasets exported as npy files (old abberior file format, pre-2025-06) as files readable by PYME, for eventual 3D surface fitting from localization clouds. 

In [ ]:
topfolder = os.path.join(parentdir, 'exampledata\\dyn1\\241101\\sample1\\cell2')

folders = os.listdir(topfolder)
folders = [folder for folder in folders if 'manual' not in folder]
roi_name = 'ROI0'

for folder in folders:
    print(folder)
    folder_date = folder.split('-')[0]
    filelist_all = os.listdir(os.path.join(topfolder,folder))
    filelist_npy = [file for file in filelist_all if file.endswith('.npy') and roi_name in file]
    for cycle, file in enumerate(filelist_npy):
        savename = f'{folder_date}-{roi_name}-cycle{cycle}.csv'
        print(savename)
        
        file = os.path.join(topfolder,folder,file)
        dataset = np.load(file)
        
        # get all (x,y,z) coodinates
        x = np.zeros((len(dataset),1))
        y = np.zeros((len(dataset),1))
        z = np.zeros((len(dataset),1))
        z1 = np.zeros((len(dataset),1))
        tid = np.zeros((len(dataset),1))
        tim = np.zeros((len(dataset),1))
        efo = np.zeros((len(dataset),1))
        for i in range(len(dataset)):
            x[i] = dataset[i][0][loc_it][2][0]
            y[i] = dataset[i][0][loc_it][2][1]
            z[i] = dataset[i][0][loc_it][2][2]
            z1[i] = dataset[i][0][1][2][2]
            efo[i] = dataset[i][0][loc_it][6]
            tid[i] = dataset[i][4]
            tim[i] = dataset[i][3]
        x = x * 1e6
        x = x.flatten()
        y = y * 1e6
        y = y.flatten()
        z = z * 1e6
        z = z * 0.7  # z scaling for immersion mismatch
        z = z.flatten()
        tid = tid.flatten()
        tim = tim.flatten()
        efo = efo.flatten()
        track_ids = list(map(int, set(tid)))
        track_ids.sort()
        data_df = pd.DataFrame(columns=['tridx', 'tim0', 'x', 'y', 'z', 'tim', 'dt', 'efo'])
        for track in track_ids[::]:
            data_row = []
            x_track = np.array([val for val,tr in zip(x,tid) if tr==track]).flatten()
            y_track = np.array([val for val,tr in zip(y,tid) if tr==track]).flatten()
            z_track = np.array([val for val,tr in zip(z,tid) if tr==track]).flatten()
            z1_track = np.array([val for val,tr in zip(z1,tid) if tr==track]).flatten()
            tim_track = np.array([val for val,tr in zip(tim,tid) if tr==track]).flatten()
            efo_track = np.array([val for val,tr in zip(efo,tid) if tr==track]).flatten()
            z_diff = z_track - z1_track[0]
            data_row.append(track)
            data_row.append(tim_track[0])
            data_row.append(x_track)
            data_row.append(y_track)
            data_row.append(z_track)
            tim_track = tim_track - tim_track[0]
            data_row.append(tim_track)
            data_row.append(np.diff(tim_track))
            data_row.append(efo_track)
            data_df = pd.concat([data_df, pd.DataFrame([data_row], columns=data_df.columns)], ignore_index=True)

        # dt filtering of tracks - remove localizations with dt larger than threshold in a rolling window
        data_df_clean = pd.DataFrame(columns=['tridx', 'tim0', 'x', 'y', 'z', 'sig', 'tim', 'dt', 'efo'])
        trs = np.arange(0,len(data_df))
        for tr in trs:
            data_row = []
            dt_track_full = np.array(data_df.iloc[tr]['dt'])*1e6
            dt_track_full = np.insert(dt_track_full,0,0)
            if len(dt_track_full) > window_pts:
                local_dt_full = []
                for i in np.arange(0, len(dt_track_full)):
                    if i > int(window_pts/2) and i < len(dt_track_full)-int(window_pts/2)-1:
                        local_dt_full.append(np.mean(dt_track_full[i-int(window_pts/2):i+int(window_pts/2)]))
                    else:
                        local_dt_full.append(np.nan)
                local_dt_full = np.array(local_dt_full)
                idx_lists = consecutive_bool(np.argwhere(local_dt_full<dt_loc_thresh).flatten())
                # loop through all accepted tracks
                for idx_list in idx_lists[:1]:
                    dt_track = dt_track_full[idx_list]
                    if len(dt_track) > len_lim:
                        x_track = np.array(data_df.iloc[tr]['x'][idx_list]).flatten()
                        y_track = np.array(data_df.iloc[tr]['y'][idx_list]).flatten()
                        z_track = np.array(data_df.iloc[tr]['z'][idx_list]).flatten()
                        sig_track = np.ones(np.shape(x_track))*10
                        tim_track = np.array(data_df.iloc[tr]['tim'][idx_list]).flatten()
                        efo_track = np.array(data_df.iloc[tr]['efo'][idx_list]).flatten()

                        data_row.append(tr)
                        data_row.append(tim_track[0])
                        data_row.append(x_track)
                        data_row.append(y_track)
                        data_row.append(z_track)
                        data_row.append(sig_track)
                        tim_track = tim_track - tim_track[0]
                        data_row.append(tim_track)
                        data_row.append(np.diff(tim_track))
                        data_row.append(efo_track)
                        data_df_clean = pd.concat([data_df_clean, pd.DataFrame([data_row], columns=data_df_clean.columns)], ignore_index=True)
                        
        x = data_df_clean['x'].explode('x')
        y = data_df_clean['y'].explode('y')
        z = data_df_clean['z'].explode('z')
        sig = data_df_clean['sig'].explode('sig')
        t = data_df_clean['tim'].explode('tim')
        dt = data_df_clean['dt'].explode('dt')
        efo = data_df_clean['efo'].explode('efo')

        data_clean = pd.DataFrame(columns=['id', 't', 'x', 'y', 'z', 'uncertainty_xy'])
        data_clean['id'] = range(len(x))
        data_clean['t'] = t
        data_clean['x'] = x*1e3
        data_clean['y'] = y*1e3
        data_clean['z'] = z*1e3
        data_clean['uncertainty_xy'] = sig
        
        data_clean.to_csv(os.path.join(topfolder,folder,savename),index=False,sep=" ")

e1
e1-ROI0-cycle0.csv
e1-ROI0-cycle1.csv
e1-ROI0-cycle2.csv
e1-ROI0-cycle3.csv
e1-ROI0-cycle4.csv
e1-ROI0-cycle5.csv
e1-ROI0-cycle6.csv
e1-ROI0-cycle7.csv
e2
e2-ROI0-cycle0.csv
e2-ROI0-cycle1.csv
e2-ROI0-cycle2.csv
e2-ROI0-cycle3.csv
e2-ROI0-cycle4.csv
